# Preprocess Chrome History

In [3]:
import pandas as pd 

In [4]:
chrome_history = pd.read_json('/bigtemp/ahs5ce/CS6501_Projects/Project1/data/raw/Takeout/Chrome/History.json', orient='index')

In [ ]:
chrome_history = chrome_history.T
chrome_history = chrome_history['Browser History']
chrome_history = pd.DataFrame(chrome_history.tolist())
chrome_history['time_usec'] = pd.to_datetime(chrome_history['time_usec'], unit='us')
chrome_history

In [8]:
chrome_history.to_csv('/bigtemp/ahs5ce/CS6501_Projects/Project1/data/processed/chrome_history.csv', index=False)

# Preprocess YouTube History

In [ ]:
# !pip install lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 27.2 MB/s eta 0:00:00a 0:00:01


In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from datetime import datetime

# Compile regex outside the loop for efficiency
video_regex = re.compile(r'watch\?v=([^&]+)')

# Read the entire HTML file (if it's very large, consider reading in chunks or filtering beforehand)
with open('/bigtemp/ahs5ce/CS6501_Projects/Project1/data/raw/Takeout/YouTube and YouTube Music/history/watch-history.html', 'r', encoding='utf-8') as f:
    html_content = f.read()

# Parse with lxml for improved performance
soup = BeautifulSoup(html_content, 'lxml')

entries = []

# Use a CSS selector to directly target watch history entries
# Each entry is in a <div> with these specific classes
outer_cells = soup.select('div.outer-cell.mdl-cell.mdl-cell--12-col.mdl-shadow--2dp')

for cell in outer_cells:
    # Narrow down to the content div that holds the video details
    content_div = cell.select_one('div.content-cell.mdl-cell.mdl-cell--6-col.mdl-typography--body-1')
    if not content_div:
        continue

    # Use a CSS selector to find the first <a> tag with a YouTube video link
    video_link = content_div.select_one('a[href*="watch?v="]')
    if video_link:
        video_url = video_link.get("href")
        video_id_match = video_regex.search(video_url)
        video_id = video_id_match.group(1) if video_id_match else None
    else:
        video_id, video_url = None, None

    # The content_div text (split by <br> tags) should contain:
    #  - "Watched"
    #  - Video title (as the text of the <a> tag)
    #  - Channel name
    #  - Timestamp (last element)
    text_parts = list(content_div.stripped_strings)
    timestamp = None
    if len(text_parts) >= 4:
        # Assume the last part is the timestamp string
        timestamp_str = text_parts[-1]
        # Replace any non-standard spaces (like narrow no-break spaces) with regular spaces
        timestamp_str = timestamp_str.replace('\u202f', ' ')
        # Remove the timezone (last token) if present, to ease parsing
        tokens = timestamp_str.split()
        if len(tokens) > 5:
            timestamp_str_mod = " ".join(tokens[:-1])
        else:
            timestamp_str_mod = timestamp_str
        try:
            # Adjust the format if your timestamps differ
            timestamp = datetime.strptime(timestamp_str_mod, "%b %d, %Y, %I:%M:%S %p")
        except Exception as e:
            # If parsing fails, fallback to storing the raw string
            timestamp = timestamp_str

    entries.append({
        "video_id": video_id,
        "video_url": video_url,
        "timestamp": timestamp
    })

# Create the DataFrame
df = pd.DataFrame(entries)
print(df.head())

      video_id                                    video_url  \
0  0R1LKPRwxR4  https://www.youtube.com/watch?v=0R1LKPRwxR4   
1  tT4br0ZKjQ0  https://www.youtube.com/watch?v=tT4br0ZKjQ0   
2  msol7hNyTkQ  https://www.youtube.com/watch?v=msol7hNyTkQ   
3  f7Cz65ef8iM  https://www.youtube.com/watch?v=f7Cz65ef8iM   
4  eKP0hOXg6Vo  https://www.youtube.com/watch?v=eKP0hOXg6Vo   

            timestamp  
0 2025-02-17 10:30:30  
1                 NaT  
2 2025-02-17 10:07:49  
3 2025-02-17 10:07:10  
4 2025-02-17 10:05:56  


In [ ]:
missing_count = df['timestamp'].isna().sum()
print("Rows with NaT in timestamp:", missing_count)
df.dropna(subset=['timestamp'], inplace=True)

Rows with NaT in timestamp: 3934


In [6]:
df.to_csv('/bigtemp/ahs5ce/CS6501_Projects/Project1/data/processed/youtube_history.csv', index=False)

In [2]:
# import pandas as pd
# from bs4 import BeautifulSoup
# import re
# from datetime import datetime

# # Read the HTML file
# with open('/bigtemp/ahs5ce/CS6501_Projects/Project1/data/raw/Takeout/YouTube and YouTube Music/history/watch-history.html', 'r', encoding='utf-8') as f:
#     soup = BeautifulSoup(f, 'html.parser')

# entries = []

# # Each watch entry is contained in a div with the class "outer-cell mdl-cell mdl-cell--12-col mdl-shadow--2dp"
# for cell in soup.find_all("div", class_="outer-cell mdl-cell mdl-cell--12-col mdl-shadow--2dp"):
#     # Find the content div that holds the details (video link, channel, and timestamp)
#     content_div = cell.find("div", class_="content-cell mdl-cell mdl-cell--6-col mdl-typography--body-1")
#     if content_div:
#         # Find the first anchor tag with a YouTube video link
#         video_a = content_div.find("a", href=re.compile(r'watch\?v='))
#         if video_a:
#             video_url = video_a.get("href")
#             # Extract the video id from the URL
#             video_id_match = re.search(r'v=([^&]+)', video_url)
#             video_id = video_id_match.group(1) if video_id_match else None
#         else:
#             video_id = None
#             video_url = None

#         # The text inside content_div is structured with <br> tags.
#         # Typically, the strings (in order) are:
#         #   1. "Watched"
#         #   2. Video title (from the video link)
#         #   3. Channel name (from the channel link)
#         #   4. Timestamp (e.g., "Feb 11, 2025, 8:44:33 PM EST")
#         text_parts = list(content_div.stripped_strings)
#         timestamp = None
#         if len(text_parts) >= 4:
#             timestamp_str = text_parts[-1]
#             # Replace any non-standard space (e.g. narrow no-break space) with a normal space
#             timestamp_str = timestamp_str.replace('\u202f', ' ')
#             # The timestamp string typically ends with a timezone (like "EST").
#             # To simplify parsing, drop the timezone.
#             # For example: "Feb 11, 2025, 8:44:33 PM EST" -> "Feb 11, 2025, 8:44:33 PM"
#             parts = timestamp_str.split()
#             if len(parts) >= 6:
#                 timestamp_str_mod = " ".join(parts[:-1])
#             else:
#                 timestamp_str_mod = timestamp_str

#             try:
#                 # Adjust the format string if your timestamps differ
#                 timestamp = datetime.strptime(timestamp_str_mod, "%b %d, %Y, %I:%M:%S %p")
#             except Exception as e:
#                 # If parsing fails, you can store the original string or handle the error
#                 timestamp = timestamp_str

#         entries.append({
#             "video_id": video_id,
#             "video_url": video_url,
#             "timestamp": timestamp
#         })

# # Create a DataFrame from the list of dictionaries
# df = pd.DataFrame(entries)
# print(df.head())

KeyboardInterrupt: 